<a href="https://colab.research.google.com/github/EstebanAG1005/Laboratorio-4-Data-Science/blob/main/Lab4_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Importación de Datos:
Utilice el conjunto de datos IMDB proporcionado por Keras. pero
esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes, utilice las 50.000
palabras más frecuentes

In [1]:
# Importando las bibliotecas y funciones necesarias.
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
# Cargando el conjunto de datos IMDB con las 50,000 palabras más frecuentes.
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

Cargando los datos...


### 2. Pre-procesamiento:

• Secuencie y rellene las críticas para que todas tengan una longitud uniforme.

• De las críticas, extraiga características (features) adicionales, por ejemplo. la
longitud de la crítica, la proporción de palabras positivas/negativas y cualquier
otra que considere pueda ser útil.

In [3]:
# Realizando preprocesamiento en los datos: ajustando la longitud de las críticas a un tamaño uniforme.
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 500  # Longitud máxima de la secuencia
X_train = sequence.pad_sequences(X_train, maxlen = max_sequence_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_sequence_length)

In [4]:
# Obteniendo el índice de palabras del conjunto de datos IMDB.
word_to_id = imdb.get_word_index()

In [5]:
# Invertiendo el índice de palabras para obtener una correspondencia del ID de la palabra a la palabra real.
id_to_word = {value: key for key, value in word_to_id.items()}

In [6]:
# Función para calcular la proporción de palabras positivas a negativas en una crítica.
def calcular_ratio_positivo_negativo(sequence):
    palabras_positivas = ["good", "excellent", "wonderful"]  # IMDB está en inglés
    palabras_negativas = ["bad", "terrible", "horrible"]

    # Convertir secuencia a palabras
    words = [id_to_word.get(i, '') for i in sequence]

    # Contar palabras positivas y negativas en el texto
    num_palabras_positivas = sum(1 for palabra in words if palabra in palabras_positivas)
    num_palabras_negativas = sum(1 for palabra in words if palabra in palabras_negativas)

    # Calcular la proporción
    if num_palabras_negativas == 0:
        return num_palabras_positivas
    else:
        return num_palabras_positivas / num_palabras_negativas

In [7]:
# Extracción de características adicionales como la longitud de la crítica y la proporción de palabras positivas/negativas.
import numpy as np

# Longitud de la crítica
lengths_train = np.array([len(seq) for seq in X_train])
lengths_test = np.array([len(seq) for seq in X_test])

# Proporción de palabras positivas/negativas (esto debe calcularse con datos previamente etiquetados)
positive_words_ratio_train = np.array([calcular_ratio_positivo_negativo(seq) for seq in X_train])
positive_words_ratio_test = np.array([calcular_ratio_positivo_negativo(seq) for seq in X_test])

In [8]:
from textblob import TextBlob

# Cálculo de polaridad y subjetividad
def extract_polarity_subj(sequence):
    words = [id_to_word.get(i, '') for i in sequence]
    review = ' '.join(words)
    blob = TextBlob(review)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

polarity_train, subj_train = zip(*[extract_polarity_subj(seq) for seq in X_train])
polarity_test, subj_test = zip(*[extract_polarity_subj(seq) for seq in X_test])

# Convertir a arrays de numpy y redimensionar
polarity_train = np.array(polarity_train).reshape(-1, 1)
polarity_test = np.array(polarity_test).reshape(-1, 1)
subj_train = np.array(subj_train).reshape(-1, 1)
subj_test = np.array(subj_test).reshape(-1, 1)


In [9]:
# Celda después de la celda anterior:

from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir secuencias en textos para el análisis de n-gramas
reviews_train = [" ".join([id_to_word.get(i, '') for i in seq]) for seq in X_train]
reviews_test = [" ".join([id_to_word.get(i, '') for i in seq]) for seq in X_test]

# Aplicar TF-IDF solo en bigramas y trigramas
vectorizer = TfidfVectorizer(ngram_range=(2, 3), max_features=5000)  # limitando a 5000 características por cuestiones de memoria y rendimiento
X_train_ngrams = vectorizer.fit_transform(reviews_train)
X_test_ngrams = vectorizer.transform(reviews_test)

# Convertir matrices dispersas en densas
X_train_ngrams = X_train_ngrams.todense()
X_test_ngrams = X_test_ngrams.todense()

In [10]:
# Redimensionando las características extraídas para que tengan la forma adecuada.
lengths_train = lengths_train.reshape(-1, 1)
lengths_test = lengths_test.reshape(-1, 1)
positive_words_ratio_train = positive_words_ratio_train.reshape(-1, 1)
positive_words_ratio_test = positive_words_ratio_test.reshape(-1, 1)

In [11]:
# Combinando las características adicionales.
additional_features_train = np.hstack([lengths_train, positive_words_ratio_train, polarity_train, subj_train, X_train_ngrams])
additional_features_test = np.hstack([lengths_test, positive_words_ratio_test, polarity_test, subj_test, X_test_ngrams])

### 3. Modelo:

• Cree un modelo LSTM que acepte las características (features) adicionales junto
con la secuencia de palabras.

• Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas
de Dropout para la regularización y tal vez alguna capa densamente conectada
después de la LSTM. (ver también la referencia al final de este documento)

In [12]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.layers import concatenate


# Entrada de secuencia
input_seq = Input(shape=(max_sequence_length,))
embedding = Embedding(50000, 128)(input_seq)

# Capa LSTM bidireccional
bi_lstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3))(embedding)
lstm = LSTM(64, dropout=0.2)(bi_lstm)

# Entrada de características adicionales
# Obtener la forma de las características adicionales (el segundo valor es el número de características)
num_additional_features = additional_features_train.shape[1]

# Modificar la entrada para las características adicionales en el modelo para que coincida con la nueva forma
input_features = Input(shape=(num_additional_features,))

# Combinar ambas entradas
merged = concatenate([lstm, input_features])
dropout = Dropout(0.2)(merged)
dense1 = Dense(64, activation='relu')(dropout)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

# Compilación
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model = Model(inputs=[input_seq, input_features], outputs=output)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Entrenar el modelo con Early Stopping y ReduceLROnPlateau
model.fit([X_train, additional_features_train], y_train,
          batch_size=64,
          epochs=15,
          verbose=2,
          validation_data=([X_test, additional_features_test], y_test),
          callbacks=[early_stopping, reduce_lr])


Epoch 1/15
391/391 - 115s - loss: 0.6899 - accuracy: 0.6438 - val_loss: 0.4097 - val_accuracy: 0.8391 - lr: 0.0010 - 115s/epoch - 295ms/step
Epoch 2/15
391/391 - 70s - loss: 0.3271 - accuracy: 0.8647 - val_loss: 0.3152 - val_accuracy: 0.8708 - lr: 0.0010 - 70s/epoch - 178ms/step
Epoch 3/15
391/391 - 56s - loss: 0.1709 - accuracy: 0.9377 - val_loss: 0.2989 - val_accuracy: 0.8714 - lr: 0.0010 - 56s/epoch - 144ms/step
Epoch 4/15
391/391 - 50s - loss: 0.1029 - accuracy: 0.9651 - val_loss: 0.3872 - val_accuracy: 0.8692 - lr: 0.0010 - 50s/epoch - 127ms/step
Epoch 5/15
391/391 - 48s - loss: 0.0648 - accuracy: 0.9802 - val_loss: 0.4317 - val_accuracy: 0.8698 - lr: 0.0010 - 48s/epoch - 124ms/step
Epoch 6/15
391/391 - 46s - loss: 0.0266 - accuracy: 0.9937 - val_loss: 0.4592 - val_accuracy: 0.8681 - lr: 2.0000e-04 - 46s/epoch - 119ms/step


### 4. Entrenamiento y Evaluación:
Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.

In [13]:
# # Entrenar el modelo con callback de Early Stopping
# model.fit([X_train, additional_features_train], y_train,
#           batch_size=64,  # Aumentado el tamaño del lote para una convergencia más estable
#           epochs=15,
#           verbose=2,
#           validation_data=([X_test, additional_features_test], y_test),
#           callbacks=[early_stopping])

In [14]:
# Evaluando el modelo y mostrando resultados.
perdida, exactitud = model.evaluate([X_test, additional_features_test], y_test,
                                    batch_size = 32,
                                    verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)


782/782 - 16s - loss: 0.4592 - accuracy: 0.8681 - 16s/epoch - 20ms/step
Pérdida de la Prueba: 0.45924100279808044
Exactitud de la Prueba (Test accuracy): 0.8680800199508667


In [15]:
# Guardando el modelo entrenado.
model.save("Sentimiento.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the training sequences.
(X_train, _), _ = tf.keras.datasets.imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from
)
# Retrieve the word index file mapping words to indices
word_index = tf.keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in X_train[0])

In [17]:
# Mostrando la secuencia decodificada.
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

### 5. Informe:


#### **Características adicionales seleccionadas y la razón de su elección**:
Se seleccionaron varias características adicionales para mejorar la capacidad del modelo de identificar el sentimiento subyacente en las críticas de películas:

1. **Longitud de la crítica**: La longitud puede ser indicativa de la naturaleza de la revisión. Las críticas extremadamente cortas o largas podrían tener un tono o sentimiento característico.
2. **Proporción de palabras positivas/negativas**: Esta característica permite cuantificar la proporción de palabras con connotaciones positivas frente a negativas en una revisión, proporcionando una métrica directa del sentimiento.
3. **Polaridad y Subjetividad**: Estos son indicadores directos del sentimiento y la objetividad de una revisión, respectivamente. Ayudan a discernir entre revisiones objetivas y opiniones sesgadas.
4. **Análisis de n-gramas**: Captura combinaciones de palabras que pueden tener significados distintos de sus componentes individuales, permitiendo al modelo identificar contextos y frases específicas que son indicativos del sentimiento.

#### **Arquitectura del modelo y las razones detrás de sus elecciones**:
La arquitectura está diseñada para manejar datos secuenciales (texto) y combinarlos con características adicionales para una predicción robusta.

- **Capa de Incrustación (Embedding)**: Transforma los índices de palabras en vectores densos de tamaño fijo. Estos vectores contienen información semántica sobre las palabras.
  
- **LSTM Bidireccional**: Esta capa captura patrones en los datos tanto en las direcciones adelante como atrás en el tiempo o secuencia. Es especialmente útil para el análisis de sentimientos, ya que el contexto puede venir antes o después de una palabra en particular.

- **Capas Densas**: Se utilizan para aprender patrones a partir de características combinadas y hacer la clasificación final. Las capas densas pueden capturar interacciones no lineales.

- **Dropout y Regularización L2**: Ayudan a prevenir el sobreajuste, asegurando que el modelo generalice bien en datos no vistos.

La elección de esta arquitectura se basa en el deseo de combinar información secuencial (texto) con características generales para obtener una comprensión más completa del sentimiento en una revisión.

#### **Resultados obtenidos y una breve comparación con el modelo simple del ejercicio anterior**:
El modelo mejorado logra un accuracy del 86.8% lo cual es una precisión notable, muestra una mejora aunque no tan significativa en comparación con el modelo anterior que tenía un 81-82%. Las características adicionales y las modificaciones en la arquitectura del modelo se realizaron con la intención de capturar más patrones y mejorar la precisión. Sin embargo, esto demuestra que el análisis de sentimientos es una tarea desafiante y que pequeñas modificaciones pueden no llevar siempre a grandes mejoras. Es crucial considerar la complejidad adicional frente a las ganancias obtenidas al hacer ajustes en el modelo.
